In [15]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import unicodedata
import re
import contractions
import string
#Gensim
import gensim
import gensim.corpora as corpora
#spacy
import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim_models

In [17]:
df = pd.read_csv('clean_tweet.csv')

In [18]:
df['Tweet'].isnull().sum()

0

In [19]:
df_no_nans=df.dropna()

In [20]:
# The LDA topic model algorithm requires a document word matrix and a dictionary as the main inputs.
# A document-term matrix is a mathematical matrix that describes the frequency of terms that occur in a collection of documents. 
# In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.
# A Dictionary is collection of all unique tokens present in documents.
# For generating the document-term matrix and dictionary, first we will convert our sentences into tokens using the fucntion mentioned below.
def generate_tokens(tweet):
    words=[]
    for word in tweet.split(' '):
    # using the if condition because we introduced extra spaces during text cleaning
        if word!='':
           words.append(word)
    return words
#storing the generated tokens in a new column named 'tokens'
df_no_nans['tokens']=df_no_nans.Tweet.apply(generate_tokens)

In [21]:
df_no_nans['tokens'][0]

['Best',
 'electric',
 'car',
 'bargain',
 'Retro',
 'anatomy',
 'best',
 'pens',
 'claudia',
 'May']

In [22]:
def create_dictionary(words):
    return corpora.Dictionary(words)
id2word=create_dictionary(df_no_nans['tokens'])
print(id2word)

Dictionary<947 unique tokens: ['Best', 'May', 'Retro', 'anatomy', 'bargain']...>


In [23]:
def create_document_matrix(tokens,id2word):
    corpus = []
    for text in tokens:
        corpus.append(id2word.doc2bow(text))
    return corpus
#passing the dataframe column having tokens and dictionary
corpus=create_document_matrix(df_no_nans['tokens'],id2word)
print(df_no_nans['tokens'][0])
print(corpus[0])

['Best', 'electric', 'car', 'bargain', 'Retro', 'anatomy', 'best', 'pens', 'claudia', 'May']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [24]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                             )

In [25]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/srv/conda/envs/notebook/lib/python3.10/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.222849 -0.042058       1        1  20.048455
6      0.087019  0.158263       2        1  17.258815
4     -0.108518 -0.162681       3        1  16.178484
9     -0.069241  0.152748       4        1  10.104499
0      0.020064 -0.120508       5        1   8.416972
3     -0.144663  0.082338       6        1   8.152161
1      0.147412 -0.084299       7        1   5.389501
8     -0.137790 -0.035004       8        1   5.326423
5     -0.040247  0.018221       9        1   4.680429
2      0.023116  0.032981      10        1   4.444261, topic_info=        Term       Freq      Total Category  logprob  loglift
12       EVs  13.000000  13.000000  Default  30.0000  30.0000
14    Ford's  12.000000  12.000000  Default  29.0000  29.0000
17        b…  12.000000  12.000000  Default  28.0000  28.0000
19      good  14.000000  14.000000  Default  27.0000  27.0000
15      It’s  12.000000  12.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
66       Air   0.497685   3.573593  Topic10  -5.0361   1.1422
22      time   0.497682  14.189494  Topic10  -5.0361  -0.2367
77  Portable   0.497679   2.245095  Topic10  -5.0361   1.6070
81    Vacuum   0.497678   2.539451  Topic10  -5.0361   1.4838
78      Pump   0.497678   4.117290  Topic10  -5.0361   1.0006

[510 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
54        8  0.631057           "
55        8  0.910061        "Fit
633       5  0.870244       "Just
375       8  0.910061   #Amazon🇨🇦
679       4  0.761195  #Bracknell
...     ...       ...         ...
796       3  0.705978           –
614       1  0.664041  “Kızılelma
118       6  0.574271           ”
472       3  0.897595           🚙
312       9  0.941828         🤣🤣🤣

[494 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 5, 10, 1, 4, 2, 9, 6, 3])